In [5]:
import rdflib
from rdflib import Graph

## 1. Importing existing RDF graph (from file)

In [6]:
from rdflib import Namespace
WE = Namespace("http://www.wikidata.org/entity/") # Wikidata entities
g1 = Graph()
g1.parse("human_kg_train.nt", format="nt")
print("Number of statements: " + str(len(g1)))
print("Some statements: ")
for s, p, o in g1.triples((WE.Q47853700, None, None)):
    print("http://www.wikidata.org/entity/Q47853700 {} {} ".format(p, o))

Number of statements: 30195
Some statements: 
http://www.wikidata.org/entity/Q47853700 http://www.wikidata.org/prop/direct/P17 http://www.wikidata.org/entity/Q408 
http://www.wikidata.org/entity/Q47853700 http://www.w3.org/2000/01/rdf-schema#label Ballangeich 


## 2. Importing existing RDF graph (remote)

In [9]:
from rdflib.namespace import DC, DCAT, DCTERMS, FOAF, OWL, PROV, RDF
g2 = Graph()
g2.parse("http://bigasterisk.com/foaf.rdf") # OR http://dig.csail.mit.edu/2008/webdav/timbl/foaf.rdf
print("Number of statements: " + str(len(g2)))
print("Some statements: ")
for s, p, o in g2.triples((None, RDF.type, FOAF.Person)):
    print("{} is a person".format(s))

Number of statements: 62
Some statements: 
Nf99be59fdd134732a101e7082cef5fd1 is a person
Nf157c82c2de84ad8b27189c7ac8699b6 is a person
http://collectivesource.com/foaf.rdf#nathan is a person
http://bigasterisk.com/foaf.rdf#drewp is a person
http://bblfish.net/people/henry/card#me is a person


## 3. Create custom RDF graph (and save to file)

In [10]:
from rdflib.namespace import DC, DCAT, DCTERMS, FOAF, OWL, PROV, RDF, RDFS, SKOS, VOID, XMLNS, XSD
from rdflib import Literal #, URIRef, BNode, 

IDS = Namespace("http://maastrichtuniversity.nl/ids/")
WP = Namespace("https://www.wikidata.org/wiki/Property:") # Wikidata properties
WT = Namespace("https://www.wikidata.org/wiki/") # Wikidata types

g3 = Graph()
g3.bind("ids", IDS)
g3.bind("wp", WP)
g3.bind("wt", WT)
g3.bind("foaf", FOAF)

g3.add((IDS.kody, RDF.type, FOAF.Person))
g3.add((IDS.vincent, RDF.type, FOAF.Person))
g3.add((IDS.kody, FOAF.name, Literal("kody moodley")))
g3.add((IDS.kody, WP.P106, WT.Q82594)) # Computer Scientist
g3.add((IDS.vincent, WP.P106, WT.Q183888)) # Software Developer
g3.add((IDS.vincent, FOAF.name, Literal("vincent emonet")))
g3.add((IDS.vincent, WP.P108, WT.Q42411212))
g3.add((IDS.kody, WP.P108, WT.Q42411212))
g3.add((IDS.kody, WP.P108, WT.Q55155670))
g3.add((IDS.kody, FOAF.knows, IDS.vincent))
g3.add((WP.P106, RDFS.label, Literal("occupation")))
g3.add((WP.P108, RDFS.label, Literal("employer")))
g3.add((WT.Q42411212, RDFS.label, Literal("Institute of Data Science at Maastricht University")))
g3.add((WT.Q55155670, RDFS.label, Literal("Faculty of Law at Maastricht University")))
g3.add((WT.Q82594, RDFS.label, Literal("Computer Scientist")))
g3.add((WT.Q183888, RDFS.label, Literal("Software Developer")))

g3.serialize(destination='custom_graph.ttl', format='turtle')
g3.serialize(destination='custom_graph.nt', format='nt')
print(g3.serialize(format="turtle").decode("utf-8"))

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix ids: <http://maastrichtuniversity.nl/ids/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix wp: <https://www.wikidata.org/wiki/Property:> .
@prefix wt: <https://www.wikidata.org/wiki/> .

ids:kody a foaf:Person ;
    foaf:knows ids:vincent ;
    foaf:name "kody moodley" ;
    wp:P106 wt:Q82594 ;
    wp:P108 wt:Q42411212,
        wt:Q55155670 .

wp:P106 rdfs:label "occupation" .

wp:P108 rdfs:label "employer" .

ids:vincent a foaf:Person ;
    foaf:name "vincent emonet" ;
    wp:P106 wt:Q183888 ;
    wp:P108 wt:Q42411212 .

wt:Q183888 rdfs:label "Software Developer" .

wt:Q55155670 rdfs:label "Faculty of Law at Maastricht University" .

wt:Q82594 rdfs:label "Computer Scientist" .

wt:Q42411212 rdfs:label "Institute of Data Science at Maastricht University" .




## 4. Querying RDF graphs with SPARQL

In [12]:
query_result1 = g1.query(
    """
        SELECT DISTINCT ?sibling1 ?sibling2
        WHERE {
              ?sibling1 <http://www.wikidata.org/prop/direct/P3373> ?sibling2 .
              ?sibling1 <http://www.wikidata.org/prop/direct/P21> <http://www.wikidata.org/entity/Q6581097> .
              ?sibling2 <http://www.wikidata.org/prop/direct/P21> <http://www.wikidata.org/entity/Q6581072> .
        }
        LIMIT 10
        
    """
)

query_result2 = g2.query(
    """
        prefix foaf: <http://xmlns.com/foaf/0.1/> 
        SELECT DISTINCT ?person
        WHERE {
              ?person rdf:type foaf:Person .
        }
         
    """
)

query_result3 = g3.query(
    """
        prefix wt: <https://www.wikidata.org/wiki/>
        prefix wp: <https://www.wikidata.org/wiki/Property:>
        SELECT DISTINCT ?person
        WHERE {
              ?person wp:P106 wt:Q82594 .
        }
        
    """
)

print("Query results for graph 1: ")
for row in query_result1:
    print("{} is the brother of his sister: {}".format(row[0], row[1]))

print()

print("Query results for graph 2: ")
for row in query_result2:
    print("%s is a person" % row)
    
print()

print("Query results for graph 3: ")
for row in query_result3:
    print("%s occupation Computer Scientist" % row)

Query results for graph 1: 
http://www.wikidata.org/entity/Q333823 is the brother of his sister: http://www.wikidata.org/entity/Q6808694
http://www.wikidata.org/entity/Q440 is the brother of his sister: http://www.wikidata.org/entity/Q507514
http://www.wikidata.org/entity/Q77109 is the brother of his sister: http://www.wikidata.org/entity/Q71407
http://www.wikidata.org/entity/Q434077 is the brother of his sister: http://www.wikidata.org/entity/Q237700
http://www.wikidata.org/entity/Q356351 is the brother of his sister: http://www.wikidata.org/entity/Q276023
http://www.wikidata.org/entity/Q60465 is the brother of his sister: http://www.wikidata.org/entity/Q70091

Query results for graph 2: 
Nf99be59fdd134732a101e7082cef5fd1 is a person
Nf157c82c2de84ad8b27189c7ac8699b6 is a person
http://collectivesource.com/foaf.rdf#nathan is a person
http://bigasterisk.com/foaf.rdf#drewp is a person
http://bblfish.net/people/henry/card#me is a person

Query results for graph 3: 
http://maastrichtunive

## 5. Reasoning over RDFS graphs

## 6. Realisation of instances into types

## 7. Identifying implict relations

In [ ]:
# from owlready2 import *
# #onto_path.append("/path/to/your/local/ontology/repository")
# onto = get_ontology("http://www.lesfleursdunormal.fr/static/_downloads/pizza_onto.owl")
# onto.load()

In [ ]:
DeductiveClosure(OWLRL_Semantics).expand(g1)